In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# %fs
# ls   - for using in DataBricks, "spark" already runned
spark = SparkSession.builder\
    .master("local")\
    .appName("lesson_16")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
json_schema = StructType([StructField("product_id", IntegerType(), True), StructField("date", DateType(), True)])

In [4]:
events_static = spark.read\
    .schema(json_schema)\
    .json("/datalake/bronze/rd_payload/2021-07-09/")

In [5]:
events_static = events_static.where(col('product_id') == 17848)

In [6]:
display(events_static)

DataFrame[product_id: int, date: date]

In [7]:
events_static.show()

+----------+----------+
|product_id|      date|
+----------+----------+
|     17848|2021-07-09|
+----------+----------+



In [8]:
events_streaming = spark.readStream\
    .schema(json_schema)\
    .option("maxFilesPerTrigger", 1)\
    .json("/datalake/bronze/rd_payload/spark_stream/")   # .kafka("...")

In [9]:
# Append mode

In [10]:
append_df = events_streaming.where(col('product_id') == 17848)

In [11]:
append_df.writeStream\
    .format("memory")\
    .queryName("event_stream_append")\
    .outputMode("append")\
    .start()

22/03/10 10:00:49 WARN streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e1e9f486-3501-4396-aa69-ad8cbc0ef90d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [12]:
# %sql
# select * from event_stream;   # it works in DataBricks
# select count(*) from event_stream;

In [13]:
# Complete mode

In [14]:
complete_df = events_streaming\
    .groupBy(col('product_id'), window(col("date"), '1 day')).count()   # windowing 1 day delay for execution (wait for grouping)

In [15]:
complete_df.writeStream\
    .format("memory")\
    .queryName("event_stream_complete")\
    .outputMode("complete")\
    .start()

22/03/10 10:01:28 WARN streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-11000624-0f08-4026-a641-668188548a6f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [16]:
# %sql
# select * from event_stream;   # it works in DataBricks
# select count(*) from event_stream;